<H3>Import Necessary Libaries</H3>

In [14]:
!pip install torch

In [18]:
import pandas as pd
import polars as pl
import numpy as np
from matplotlib import pyplot as plt
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split

import torch
import pytorch_lightning as pl
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

OSError: [WinError 126] The specified module could not be found. Error loading "C:\Python311\Lib\site-packages\torch\lib\shm.dll" or one of its dependencies.

<H3>Load In Datasets</H3>

In [2]:
df_test = pd.read_csv("creditcard.csv")
df = pl.read_csv("creditcard.csv", schema_overrides={"Time": pl.Utf8}).drop("Time")

In [3]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

<H3>Data Exploration</H3>

In [ ]:
profile = ProfileReport(df_test)
profile

<H3>Data Preprocessing</H3>

In [5]:
X = df.select(pl.all().exclude("Class")).to_numpy()
y = df.select("Class").to_numpy().flatten()

<H4>SMOTE</H4>

In [6]:
from imblearn.over_sampling import SMOTE

In [7]:
smote = SMOTE()
X_resample, y_resample = smote.fit_resample(X, y)

In [9]:
print('Number of total transactions before SMOTE upsampling: ', len(y), '...after SMOTE upsampling: ', len(y_resample))
print('Number of fraudulent transactions before SMOTE upsampling: ', np.count_nonzero(y==1), 
      '...after SMOTE upsampling: ', np.sum(y_resample[y_resample==1]))

Number of total transactions before SMOTE upsampling:  284807 ...after SMOTE upsampling:  568630
Number of fraudulent transactions before SMOTE upsampling:  492 ...after SMOTE upsampling:  284315


In [12]:
X_resample.shape

(568630, 29)

<H4>Autoencoder</H4>

In [ ]:
class AutoEncoder(pl.LightningModule):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 12),
            nn.ReLU(),
            nn.Linear(12, 3)  # Bottleneck layer
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(),
            nn.Linear(12, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 28 * 28),
            nn.Sigmoid()  # Output layer
        )

        def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def predict_step(self, predict_batch, batch_idx):
        x, _ = predict_batch
        y = self(x)
        y = torch.softmax(y,-1)
        return y
    
    def training_step(self, batch, batch_idx):
        x, _ = batch
        x_hat = self(x)
        loss = nn.MSELoss()(x_hat, x)
        self.log('train_loss', loss, on_step=False, on_epoch=True)
        return loss
    
    def validation_step(self, val_batch, batch_idx):
        x, _ = val_batch
        x_hat = self(x)
        loss = nn.MSELoss()(x_hat, x)
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        return loss
    
    def test_step(self, test_batch, batch_idx):
        x, y = test_batch
        y_pred = self(x)
        loss = nn.MSELoss()(x_hat, x)
        self.log('test_loss', loss, on_step=False, on_epoch=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

In [ ]:
model = Autoencoder()

<H3>Modeling</H3>

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_resample, y_resample, test_size=0.2)

<H4>Random Forest</H4>

In [ ]:
decision_tree = DecisionTreeClassifier()

<H4>Neural Network</H4>

In [ ]:
class NeuralNet(pl.LightningModule):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(29, 16),
            nn.ReLU(),
            nn.Linear(16, 24),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(24, 24),
            nn.ReLU(),
            nn.Linear(24, 1),
            nn.Sigmoid(),
        )
        self.loss_fn = nn.BCELoss()

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def training_step(self, batch):
        x, y = batch
        y_hat = self.forward(x).squeeze()
        loss = self.loss_fn(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def test_step(self, batch):
        X, y = batch
        y_hat = self(X)
        y_pred = (y_hat > 0.5).float()
        accuracy = (y_pred.squeeze() == y).float().mean()
        self.log("test_accuracy", accuracy)
        return accuracy


if __name__ == "__main__":
    torch.set_float32_matmul_precision('medium')
    torch.device("cuda" if torch.cuda.is_available() else "cpu")

    df = polars.read_csv("creditcard.csv", schema_overrides={
                         "Time": polars.Utf8}).drop("Time")

    X = df.select(polars.all().exclude("Class")).to_numpy()
    y = df.select("Class").to_numpy().flatten()

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2
    )

    train_dataset = TensorDataset(
        torch.FloatTensor(X_train), torch.FloatTensor(y_train)
    )
    test_dataset = TensorDataset(
        torch.FloatTensor(X_test), torch.FloatTensor(y_test))

    train_loader = DataLoader(
        train_dataset, batch_size=64, shuffle=True, num_workers=16)

    test_loader = DataLoader(test_dataset, batch_size=64, num_workers=16)

    model = NeuralNet()

    trainer = pl.Trainer(max_epochs=5)

    trainer.fit(model, train_loader)

    trainer.test(model, test_loader)

In [2]:
if __name__ == "__main__":
    torch.set_float32_matmul_precision('medium')
    torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_dataset = TensorDataset(
        torch.FloatTensor(X_train), torch.FloatTensor(y_train)
    )
    test_dataset = TensorDataset(
        torch.FloatTensor(X_test), torch.FloatTensor(y_test))

    train_loader = DataLoader(
        train_dataset, batch_size=64, shuffle=True, num_workers=16)

    test_loader = DataLoader(test_dataset, batch_size=64, num_workers=16)

    model = NeuralNet()

    trainer = pl.Trainer(max_epochs=5)

    trainer.fit(model, train_loader)

    trainer.test(model, test_loader)

NameError: name 'NeuralNetwork' is not defined

<h1>------------------------------------------------------------------------------------------</h1>

In [7]:
import torch
import lightning.pytorch as pl
from torchinfo import summary
from torchview import draw_graph
import torchmetrics
import torchvision
import pyspark
from sklearn.tree import DecisionTreeClassifier

In [11]:
df = pl.read_csv("creditcard.csv", schema_overrides={"Time": pl.Utf8}).drop("Time")

In [12]:
df.head()

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.5516,-0.617801,-0.99139,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1.191857,0.266151,0.16648,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.16717,0.125895,-0.008983,0.014724,2.69,0
-1.358354,-1.340163,1.773209,0.37978,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.52498,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.1083,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.5,0
-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.11967,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.20601,0.502292,0.219422,0.215153,69.99,0
